In [15]:
import numpy as np
import os, sys, time
import dask.array as da
import dask_jobqueue
from dask_jobqueue import LSFCluster
from fish.util.distributed import get_jobqueue_cluster
from dask.distributed import Client
import B.general_admin as ga

In [54]:
def load_data():
    ''' hardcoded function to load data (≈6GB) ''' 
    folder = '/nrs/ahrens/Virginia_nrs/25_LS_wG/190703_HuCH2B_gCaMP7F_8dpf_forG/exp0/sca/'
    fname = folder + 'data_ntk_demeaned.npy'
    return np.load(fname)

def construct_adiff(dummy_var, block_id):
    ''' I haad trouble running the code without using the blockid'''
    j = block_id[0] # neuron number
    data_ntk = load_data()
    n, t, k = data_ntk.shape # n = 31305, t = 50, k ≈ 800 
    jtk = data_ntk[j] # load data from neuron j
    mat_nt2 = np.zeros([n, t**2]) # prepare result matrix
    for i in range(n): # loop over all neurons...
        if np.mod(i, 5000) == 0:
            print('n: {0}'.format(i))
        itk = data_ntk[i]
        ai = itk.dot(jtk.T)/k # construct TxT covariance matrix      
        ai_diff = (ai - ai.T).reshape([1, t**2]) # construct small cdiff and reshape into 1x(T^2)
        mat_nt2[i] = ai_diff
    return np.expand_dims(mat_nt2.dot(mat_nt2.T),0)

In [4]:
cluster = get_jobqueue_cluster()
client = Client(cluster)
client

Client Scheduler: tcp://10.36.111.13:41403 Dashboard: http://10.36.111.13:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [38]:
cluster.start_workers(3)

In [50]:
# test that function works
# %time y = construct_adiff(0, [0]);
# print(y.shape)

In [55]:
n = 31305 # number of neurons (to determine the size of the result (nn x n x n) which we sum over the first dimension)
nn = 2 # subset of neurons to test the code over
%time x = da.arange(nn, chunks = 1).map_blocks(construct_adiff,\
                                              dtype='float32', new_axis=[1,2], chunks = (nn,n,n)).sum(0)

CPU times: user 68.3 ms, sys: 2.27 s, total: 2.34 s
Wall time: 2.25 s


In [56]:
x

dask.array<sum-aggregate, shape=(31305, 31305), dtype=float32, chunksize=(31305, 31305)>

In [57]:
t0 = time.time()
%time result = x.compute()
print(time.time()-t0)

Worker tcp://10.36.60.25:45354 restart in Job 62438657. This can be due to memory issue.
Worker tcp://10.36.60.32:37084 restart in Job 62438656. This can be due to memory issue.
Worker tcp://10.36.60.25:46135 restart in Job 62438657. This can be due to memory issue.
Worker tcp://10.36.60.31:44474 restart in Job 62438655. This can be due to memory issue.
Worker tcp://10.36.60.32:43358 restart in Job 62438656. This can be due to memory issue.


KilledWorker: ("('arange-construct_adiff-sum-041104f3a876a585627c0a1af74c3673', 0, 0, 0)", <Worker 'tcp://10.36.60.25:46135', memory: 0, processing: 1>)

218.13727498054504


Worker tcp://10.36.60.25:37409 restart in Job 62438657. This can be due to memory issue.


In [43]:
result.shape

(31305, 31305)

Worker tcp://10.36.60.25:39061 restart in Job 62438657. This can be due to memory issue.


In [37]:
cluster.stop_all_jobs()